In [1]:
import os
import yaml
import shutil
from pathlib import Path
from pprint import pprint

%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: 
        - '/storage/vbutoi/scratch/ESE/inference/10_26_24_OCTA_Benchmark'
        - '/storage/vbutoi/scratch/ESE/inference/10_26_24_ISLES_Benchmark'
        - '/storage/vbutoi/scratch/ESE/inference/10_26_24_Roads_FULLRES_Benchmark'
        - '/storage/vbutoi/scratch/ESE/inference/10_26_24_WMH_Benchmark'
    inference_group: 'Sweep_Temperature'

## Setup

In [3]:
# Build a dictionary to store the inference info.
log_cfg = results_cfg["log"] 

# Skip over metadata folders
skip_log_folders = [
    "debug",
    "wandb", 
    "submitit", 
]
# We need to get the roots and inference groups from the log_cfg.
log_roots = log_cfg["root"]
log_inference_groups = log_cfg["inference_group"]

if isinstance(log_roots, str):
    log_roots = [log_roots]
if isinstance(log_inference_groups, str):
    log_inference_groups = [log_inference_groups]

## Get the log dir folders.

In [4]:

# Gather inference log paths.
all_inference_log_paths = []
for root in log_roots:
    for inf_group in log_inference_groups:
        inf_group_dir = root + "/" + inf_group
        group_folders = os.listdir(inf_group_dir)
        # If 'submitit' is in the highest dir, then we don't have subdirs (new folder structure).
        if "submitit" in group_folders:
            # Check to make sure that this log wasn't the result of a crash.
            all_inference_log_paths.append(Path(inf_group_dir))
        # Otherwise, we had separated our runs in 'log sets', which isn't a good level of abstraction.
        # but it's what we had done before.
        else:
            for sub_exp in group_folders:
                sub_exp_log_path = inf_group_dir + "/" + sub_exp
                # TODO: FIX THIS, I HATE THE SKIP_LOG_FOLDER PARADIGM.
                # Verify that it is a folder and also that it is not in the skip_log_folders.
                if os.path.isdir(sub_exp_log_path) and sub_exp not in skip_log_folders:
                    sub_exp_group_folders = os.listdir(sub_exp_log_path)
                    # If 'submitit' is in the highest dir, then we don't have subdirs (new folder structure).
                    if "submitit" in sub_exp_group_folders:
                        # Check to make sure that this log wasn't the result of a crash.
                        all_inference_log_paths.append(Path(sub_exp_log_path))

In [ ]:
all_inference_log_paths

### Get the actual config files.

In [6]:
config_files = []

for log_dir in all_inference_log_paths:
    for log_set in log_dir.iterdir():
        # TODO: FIX THIS, I HATE THE SKIP_LOG_FOLDER PARADIGM.
        # Verify that log_set is a directory and that it's not in the skip_log_folders.
        if log_set.is_dir() and log_set.name not in skip_log_folders:
            # Load the metadata file (json) and add it to the metadata dataframe.
            logset_config_dir = log_set / "config.yml"
            config_files.append(logset_config_dir)

In [ ]:
config_files

In [8]:
# load the file config path and print the contents.
first_config = config_files[0]

In [19]:
def modify_yml(file_path):
    # Step 1: Load the .yml file into a dictionary
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    
    # We want to modify the exp_root
    old_exp_root = data['experiment']['exp_root']
    new_root = old_exp_root.replace("10_25_24", "10_26_24")

    # Place it in
    data['experiment']['exp_root'] = new_root
    data['log']['root'] = new_root

    backup_path = Path(str(file_path).replace("config.yml", "backup_config.yml"))

    # Step 3: Backup the old .yml file
    shutil.move(file_path, backup_path)
    
    # Step 4: Write the new .yml file
    with open(file_path, 'w') as file:
        yaml.dump(data, file)

    print("Moved {} -> {}".format(file_path, backup_path))

In [ ]:
for cfg_path in config_files:
    modify_yml(cfg_path)